# Libraries

In [1]:
import os
import time

import tfx
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft

from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tfx.components import CsvExampleGen
from tfx.components import SchemaGen
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components import ExampleValidator
from tfx.components import Transform

# Running Data Cleaning File

In [2]:
!python clean_data.py

clean_data.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Wages'] = df['Wages'].str.replace(',', '')
clean_data.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Wages'] = df['Wages'].str.replace('€ ', '')
clean_data.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

# Artifact, Serving Model, Data Locations

In [3]:
ARTIFACT_STORE = os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'TFX_football_manager_2022', 'Artifacts')
SERVING_MODEL_DIR = os.path.join(os.sep, 'Users', 'CarlosMonsivais', 'Desktop', 'TFX_football_manager_2022', 'Model')
DATA_ROOT = '/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Clean_Data'

print('Artifact Storage: {}'.format(ARTIFACT_STORE))
print('Serving Model Storage: {}'.format(SERVING_MODEL_DIR))
print('Data Root: {}'.format(DATA_ROOT))

Artifact Storage: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts
Serving Model Storage: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Model
Data Root: /Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Clean_Data


# Pipeline Names

In [4]:
PIPELINE_NAME = 'tfx-football-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))

# Creating a directory in the Artifact Storage variable where the pipline metadata will be stored using this timestamp.
os.makedirs(PIPELINE_ROOT, exist_ok = True)

print('Created a directory here:{} \nwhere the pipleine metadata artifacts will be stored.'.format(PIPELINE_ROOT))

Created a directory here:/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457 
where the pipleine metadata artifacts will be stored.


# Only for Jupyter Notebook Interactivity

In [5]:
context = InteractiveContext(pipeline_name = PIPELINE_NAME,
                             pipeline_root = PIPELINE_ROOT,
                             metadata_connection_config = None)

# Reading in Data

In [6]:
input_config = example_gen_pb2.Input(splits=[example_gen_pb2.Input.Split(name = 'clean_football_manager_22', 
                                                                         pattern = '*.csv')])

In [7]:
# Doing an 80:20 Train Eval Split
output_config = example_gen_pb2.Output(split_config = example_gen_pb2.SplitConfig(splits=[example_gen_pb2.SplitConfig.Split(name = 'train', hash_buckets = 4),
                                                                                          example_gen_pb2.SplitConfig.Split(name = 'eval', hash_buckets = 1)]))

In [8]:
example_gen = tfx.components.CsvExampleGen(input_base = DATA_ROOT,
                                           input_config = input_config,
                                           output_config = output_config)

In [9]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:clean_football_manager_22,num_files:1,total_bytes:46907884,xor_checksum:1643138088,sum_checksum:1643138088"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [10]:
examples_uri = example_gen.outputs['examples'].get()[0].uri

In [11]:
examples_uri

'/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/CsvExampleGen/examples/1'

In [13]:
# Shows features
tfrecord_filenames = [os.path.join(examples_uri, 'train', name) for name in os.listdir(os.path.join(examples_uri, 'train'))]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type = "GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Tackling: [13.0]
Composure: [10]
Communication: []
Long Throws: [3.0]
Unique ID: [62127037]
Off the Ball: [16]
Club: [b'Lazio']
One on Ones: []
Wages: [113890.0]
Technique: [16]
Agility: [13]
Flair: [18]
First Touch: [16]
Heading: [17.0]
Rushing Out (Tendency): []
Potential: [85.0]
Work Rate: [15]
Natural Fitness: [20]
Command of Area: []
Corners: [3.0]
Strength: [17]
Nation: [b'Serbia']
Age: [26]
Teamwork: [16]
Acceleration: [13]
Punching (Tendency): []
Stamina: [18]
Anticipation: [15]
Concentration: [10]
Handling: []
Reflexes: []
Bravery: [17]
Sell value: [73441416.0]
Throwing: []
Foot: [b'Right']
Kicking: []
Penalty Taking: [13]
Weight: [82]
Crossing: [6.0]
Determination: [17]
Aerial Reach: []
Jumping Reach: [17]
Dribbling: [14.0]
Length: [191]
Ability: [82.0]
Decisions: [13]
Marking: [9.0]
Free Kick Taking: [15]
Aggression: [14]
Contract end: [b'2024-06-30']
Long Shots: [15.0]
Caps: [22.0]
Passing: [16]
Goals: [3.0]
Name: [b'Sergej Milinkovic-Savic']
Finishing: [15.0]
Leadership: [

# StaisticsGen Component

In [14]:
statistics_gen = tfx.components.StatisticsGen(examples = example_gen.outputs['examples'])

In [15]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [16]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


# SchemaGen

In [17]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=False)

In [18]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

In [19]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Ability',FLOAT,required,single,-
'Acceleration',INT,required,single,-
'Aerial Reach',FLOAT,required,,-
'Age',INT,required,single,-
'Aggression',INT,required,single,-
...,...,...,...,...
'Unique ID',INT,required,single,-
'Vision',INT,required,single,-
'Wages',FLOAT,required,,-


/opt/anaconda3/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Foot',"'Both', 'Left', 'Right'"


## Updating SchemaGen File (In case you want to make any changes to the file.)

In [ ]:
# You can chage this file to customize it as you want it for your schema.
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [ ]:
# tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))   (for categorical variable)
tfdv.set_domain(schema, 'Ability',  schema_pb2.IntDomain(name='Ability', min=0, max=20, ))

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

In [ ]:
schema_importer = ImporterNode(instance_name = 'Schema_Importer',
                               source_uri = schema_dir,
                               artifact_type=tfx.types.standard_artifacts.Schema,
                               reimport=False)

In [ ]:
context.run(schema_importer)

In [ ]:
context.show(schema_importer.outputs['result'])

# Example Validator (Anomaly Detection)

In [20]:
# Only use if the schema_importer is one that you edited 

# example_validator = ExampleValidator(instance_name = "Data_Validation",
#                                      statistics = statistics_gen.outputs['statistics'],
#                                      schema = schema_importer.outputs['result'])


example_validator = ExampleValidator(instance_name = "Data_Validation",
                                     statistics = statistics_gen.outputs['statistics'],
                                     schema = schema_gen.outputs['schema'])

In [21]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/ExampleValidator.Data_Validation/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [22]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")

In [23]:
train_anomalies_filename

'/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/ExampleValidator.Data_Validation/anomalies/4/train/anomalies.pbtxt'

In [24]:
context.show(example_validator.outputs['output'])

# Transform

In [31]:
TRANSFORM_MODULE = 'transform.py'
!cat {TRANSFORM_MODULE}
#!python transform.py

import tensorflow as tf
import tensorflow_transform as tft
import data_features

def _fill_in_missing(x):
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), default_value), axis=1)

def preprocessing_fn(inputs):
  outputs = {}

  # Scale numerical features.
  for key in data_features.NUMERIC_FEATURE_KEYS:
    outputs[data_features.transformed_feature_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))


  # Generate vocabularies and maps categorical features.
  for key in data_features.CATEGORICAL_FEATURE_KEYS:
    outputs[data_features.transformed_feature_name(key)] = inputs[key]


  return outputs

In [32]:
example_gen.outputs['examples']

Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:clean_football_manager_22,num_files:1,total_bytes:46907884,xor_checksum:1643138088,sum_checksum:1643138088"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
)

In [33]:
transform = Transform(examples = example_gen.outputs['examples'],
                      schema = schema_gen.outputs['schema'],
                      module_file = TRANSFORM_MODULE)                      

In [34]:
example_gen.outputs['examples']

Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:clean_football_manager_22,num_files:1,total_bytes:46907884,xor_checksum:1643138088,sum_checksum:1643138088"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
)

In [35]:
context.run(transform)

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 11
        type_id: 13
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/Transform/transform_graph/7"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 12
        type_id: 5
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/Transform/transformed_examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 13
        type_id: 14
        uri: "/Users/CarlosMonsivais/Desktop/TFX_football_manager_2022/Artifacts/tfx-football-pipeline/20220125_111457/Transform/updated_analyzer_cache/7"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
        ))